In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [80]:
data1 = [0.1, 0.2, 0.2, 0.12, 0.2, 0.13,
         0.15, 0.25, 0.18, 0.08, 0.19, 0.12]
data2 = [0.6, 0.52, 0.45]
data3 = [0.68, 0.28,
         0.75, 0.2]

rawMaterials = ['CrudeOil1', 'CrudeOil2']
finalProducts = ['PremiumPetrol', 'RegularPetrol', 'JetFuel', 
                 'FuelOil', 'LubeOil']
productProfit = dict(zip(finalProducts, [7, 6, 4, 3.5, 1.5]))


distilOutputs = ['LightNaphtha', 'MediumNaphtha', 'HeavyNaphtha', 'LightOil', 'HeavyOil', 'Residuum']
naphthas = ['LightNaphtha', 'MediumNaphtha', 'HeavyNaphtha']
oils = ['LightOil', 'HeavyOil']
oils_plus = ['LightOil', 'HeavyOil', 'CrackedOil', 'Residuum']
reformOutputs = ['ReformedGasoline']
crackiOutputs = ['CrackedOil', 'CrackedGasoline']

distilYields = dict(zip(it.product(rawMaterials, distilOutputs), data1))
reformYields = dict(zip(it.product(distilOutputs[:3], reformOutputs), data2))
crackiYields = dict(zip(it.product(distilOutputs[3:5], crackiOutputs), data3))

propor = dict(zip(['LightOil', 'CrackedOil', 'HeavyOil', 'Residuum'],
                   [10/18, 4/18, 3/18, 1/18]))
qualit = dict(zip(['PremiumPetrol', 'RegularPetrol'],
                  [94, 84]))
octane = dict(zip(['LightNaphtha', 'MediumNaphtha', 'HeavyNaphtha', 'ReformedGasoline', 'CrackedGasoline'],
                  [90, 80, 70, 115, 105]))
pressu = dict(zip(['LightOil', 'CrackedOil', 'HeavyOil', 'Residuum'],
                  [1.0, 1.5, 0.6, 0.05]))

all_mat = rawMaterials + finalProducts + distilOutputs + reformOutputs + crackiOutputs
used_to = [('LightNaphtha', 'PremiumPetrol'),
            ('MediumNaphtha', 'PremiumPetrol'),
            ('HeavyNaphtha', 'PremiumPetrol'),
            ('ReformedGasoline', 'PremiumPetrol'),
            ('CrackedGasoline', 'PremiumPetrol'),
            ('LightNaphtha', 'RegularPetrol'),
            ('MediumNaphtha', 'RegularPetrol'),
            ('HeavyNaphtha', 'RegularPetrol'),
            ('ReformedGasoline', 'RegularPetrol'),
            ('CrackedGasoline', 'RegularPetrol'),
            ('LightOil', 'JetFuel'),
            ('HeavyOil', 'JetFuel'),
            ('Residuum', 'JetFuel'),
            ('CrackedOil', 'JetFuel'),
            ('LightOil', 'Cracked'),
            ('HeavyOil', 'Cracked'),
            ('Residuum', 'LubeOil')] + list(reformYields.keys())

used_in = {'LightNaphtha': ['PremiumPetrol', 'RegularPetrol', 'ReformedGasoline'],
           'MediumNaphtha': ['PremiumPetrol', 'RegularPetrol', 'ReformedGasoline'],
           'HeavyNaphtha': ['PremiumPetrol', 'RegularPetrol', 'ReformedGasoline'],
           'LightOil': ['Cracked', 'JetFuel'],
           'HeavyOil': ['Cracked', 'JetFuel'],
           'CrackedOil': ['JetFuel'],
           'Residuum': ['JetFuel', 'LubeOil'],
           'CrackedGasoline': ['PremiumPetrol', 'RegularPetrol'],
           'ReformedGasoline': ['PremiumPetrol', 'RegularPetrol']}

ingred = {'PremiumPetrol': ['LightNaphtha', 'MediumNaphtha', 'HeavyNaphtha', 'ReformedGasoline', 'CrackedGasoline'],
          'RegularPetrol': ['LightNaphtha', 'MediumNaphtha', 'HeavyNaphtha', 'ReformedGasoline', 'CrackedGasoline'],
          'JetFuel': ['LightOil', 'HeavyOil', 'Residuum', 'CrackedOil']}



In [81]:
model = gp.Model('Refinery Optimization')

# add vars
x = model.addVars(all_mat,  
                  name='x')
y = model.addVars(used_to,
                  name='y')

    # set upper bound (availability of raw materials)
x['CrudeOil1'].ub = 20000
x['CrudeOil2'].ub = 30000
    
    # set bounds for Lube Oil production (constraint 6 in the book)
x['LubeOil'].ub = 1000
x['LubeOil'].lb = 500

# objective function
model.setObjective((gp.quicksum(productProfit[p]*x[p] for p in finalProducts)), GRB.MAXIMIZE)

# add constraints

    # capacities (constraints 3 to 5 in the book)
model.addConstr((x['CrudeOil1'] + x['CrudeOil2'] <= 45000),
                name='distillation')
model.addConstr((gp.quicksum(y[n,'ReformedGasoline'] for n in naphtas) <= 10000),
                name='reforming')
model.addConstr((gp.quicksum(y[o,'Cracked'] for o in oils) <= 8000),
                name='cracking') 

    # conservation
for p in distilOutputs:
    model.addConstr((x[p] == gp.quicksum(distilYields[m,p]*x[m] for m in rawMaterials)), name='dist_'+p)
    
p = 'ReformedGasoline'
model.addConstr((x[p] == gp.quicksum(reformYields[n,p]*y[n,p] for n in naphthas)), name='refo_'+p)

for p in crackiOutputs:
    model.addConstr((x[p] == gp.quicksum(crackiYields[o,p]*y[o,'Cracked'] for o in oils)), name='crac_'+p)

p = 'LubeOil'
model.addConstr((x[p] == 0.5*y[('Residuum', p)]), name='lube')

for p in naphthas:
    model.addConstr((x[p] == gp.quicksum(y[p,i] for i in used_in[p])), name=p)
    
for p in oils_plus:
    model.addConstr((x[p] == gp.quicksum(y[p,i] for i in used_in[p]) + propor[p]*x['FuelOil']), name=p)
    
p = 'CrackedGasoline'
model.addConstr((x[p] == gp.quicksum(y[p,i] for i in used_in[p])), name=p)
p = 'ReformedGasoline'
model.addConstr((x[p] == gp.quicksum(y[p,i] for i in used_in[p])), name=p)

for p in ['PremiumPetrol', 'RegularPetrol', 'JetFuel']:
    model.addConstr((x[p] == gp.quicksum(y[i,p] for i in ingred[p])), name=p)

model.addConstr((x['PremiumPetrol'] >= 0.4*x['RegularPetrol']), name='40perc')

    # quality
for p in ['PremiumPetrol', 'RegularPetrol']:
    model.addConstr((qualit[p]*x[p] <= gp.quicksum(octane[i]*y[i,p] for i in ingred[p])), name='octa_'+p)

p = 'JetFuel'
model.addConstr((x[p] >= gp.quicksum(pressu[i]*y[i,p] for i in ingred[p])), name='pres_'+p)

model.update()
model.write('Refinery Optimization.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 29 rows, 36 columns and 106 nonzeros
Model fingerprint: 0x59522f92
Coefficient statistics:
  Matrix range     [5e-02, 1e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [5e+02, 3e+04]
  RHS range        [8e+03, 5e+04]
Presolve removed 13 rows and 14 columns
Presolve time: 0.01s
Presolved: 16 rows, 22 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1884516e+06   6.044540e+04   0.000000e+00      0s
      14    2.1136513e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.02 seconds
Optimal objective  2.113651348e+05


In [85]:
print('Profit of ${:.2f}'.format(model.ObjVal))

print('Quantities of materials and final products:')
for p in x.keys():
    print('{:19} {:.1f}'.format(p, x[p].x))

Profit of $211365.13
Quantities of materials and final products:
CrudeOil1           15000.0
CrudeOil2           30000.0
PremiumPetrol       6817.8
RegularPetrol       17044.4
JetFuel             15156.0
FuelOil             0.0
LubeOil             500.0
LightNaphtha        6000.0
MediumNaphtha       10500.0
HeavyNaphtha        8400.0
LightOil            4200.0
HeavyOil            8700.0
Residuum            5550.0
ReformedGasoline    2433.1
CrackedOil          5706.0
CrackedGasoline     1936.0
